In [ ]:
# =============================================================================
# DATABASE SCHEMA VERIFICATION - JUPYTER NOTEBOOK VERSION
# Generated: 2026-02-16
# Purpose: Verify that your database schema matches the SQL definition files
# =============================================================================
#
# INSTRUCTIONS:
# 1. Copy each cell below into a separate Jupyter notebook cell
# 2. Run the cells in order
# 3. Look for ✓ PASS or ✗ FAIL in the results
# 4. Any ✗ FAIL indicates a schema mismatch that needs investigation
#
# =============================================================================

# =============================================================================
# CELL 1: Setup - Database Connection
# Run this first to establish connection to your database
# =============================================================================


In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
# Environment variable management
# python-dotenv: Loads database credentials from .env file (keeps passwords out of code)
from dotenv import load_dotenv
import os

In [ ]:


load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT')
)

# SQLAlchemy engine (needed for pd.read_sql)
engine = create_engine(
    f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@"
    f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)

print("✓ Connection configured!")

In [ ]:
# =============================================================================
# CELL 2: Test Connection
# Verify the connection works before running verification queries
# =============================================================================

query = "SELECT version();"
result = pd.read_sql(query, engine)
print("✓ Connection successful!")
print(result)


In [ ]:
# =============================================================================
# CELL 3: Table Existence Check
# Expected: 7 tables
# =============================================================================

query = """
SELECT 
    'Table Existence Check' as verification_type,
    table_name,
    CASE 
        WHEN table_name IN ('images', 'ground_truth_history', 'prompts', 
                           'analysis_runs', 'llm_responses', 'predictions', 
                           'clustering_results')
        THEN '✓ EXPECTED'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.tables
WHERE table_schema = 'public' 
  AND table_type = 'BASE TABLE'
ORDER BY table_name;
"""

df = pd.read_sql(query, engine)
print("=== TABLE EXISTENCE CHECK ===")
display(df)

# Quick summary
query_summary = """
SELECT 
    'Total Tables' as metric,
    COUNT(*) as actual_count,
    7 as expected_count,
    CASE WHEN COUNT(*) = 7 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.tables
WHERE table_schema = 'public' 
  AND table_type = 'BASE TABLE';
"""
df_summary = pd.read_sql(query_summary, engine)
display(df_summary)


In [ ]:
# =============================================================================
# CELL 4: Images Table - Column Structure
# Verify all columns exist with correct data types
# =============================================================================

query = """
SELECT 
    'images table columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    column_default,
    CASE 
        WHEN column_name = 'image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'source' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'source_image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'filename' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'file_path' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'date_added' AND data_type = 'timestamp without time zone' THEN '✓ CORRECT'
        WHEN column_name = 'total_size_bytes' AND data_type = 'integer' AND is_nullable = 'YES' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'images'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== IMAGES TABLE - COLUMN STRUCTURE ===")
display(df)


In [ ]:
# =============================================================================
# CELL 5: Ground Truth History Table - Column Structure
# =============================================================================

query = """
SELECT 
    'ground_truth_history columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name = 'history_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'label_name' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'value' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'changed_at' AND data_type = 'timestamp without time zone' THEN '✓ CORRECT'
        WHEN column_name = 'reason' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'is_current' AND data_type = 'boolean' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'ground_truth_history'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== GROUND_TRUTH_HISTORY TABLE - COLUMN STRUCTURE ===")
display(df)


In [ ]:
# =============================================================================
# CELL 6: Prompts Table - Column Structure
# =============================================================================

query = """
SELECT 
    'prompts table columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name = 'prompt_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'prompt_name' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'prompt_text' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'created_at' AND data_type = 'timestamp without time zone' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'prompts'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== PROMPTS TABLE - COLUMN STRUCTURE ===")
display(df)



In [ ]:
# =============================================================================
# CELL 7: Analysis Runs Table - Column Structure
# This includes original columns + 10 clustering columns from migration
# =============================================================================
query = """
SELECT 
    'analysis_runs columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        -- Original columns from create_tables.sql
        WHEN column_name = 'analysis_run_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'run_timestamp' AND data_type = 'timestamp without time zone' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'analysis_type' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'model_name' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'python_script' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'model_version' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'hyperparameters' AND data_type = 'jsonb' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'notes' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'images_processed' AND data_type = 'integer' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'start_time' AND data_type = 'timestamp without time zone' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'duration_seconds' AND data_type = 'real' AND is_nullable = 'YES' THEN '✓ CORRECT'
        -- Clustering columns from alter_analysis_runs_clustering.sql
        WHEN column_name = 'autoencoder_name' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'autoencoder_implementation' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'autoencoder_file' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'autoencoder_params' AND data_type = 'jsonb' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'dim_reduction_name' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'dim_reduction_implementation' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'dim_reduction_params' AND data_type = 'jsonb' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'clustering_name' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'clustering_implementation' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'clustering_params' AND data_type = 'jsonb' AND is_nullable = 'YES' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED OR WRONG TYPE'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'analysis_runs'
ORDER BY ordinal_position;
"""
df = pd.read_sql(query, engine)
print("=== ANALYSIS_RUNS TABLE - COLUMN STRUCTURE ===")
print("(Should have 21 columns: 11 original + 10 clustering)")
display(df)

# Column count check
query_count = """
SELECT 
    'analysis_runs column count' as metric,
    COUNT(*) as actual_count,
    21 as expected_count,
    CASE WHEN COUNT(*) = 21 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'analysis_runs';
"""
df_count = pd.read_sql(query_count, engine)
display(df_count)

In [ ]:
# =============================================================================
# CELL 8: LLM Responses Table - Column Structure
# =============================================================================

query = """
SELECT 
    'llm_responses columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name = 'response_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'analysis_run_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'prompt_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'parsed_response' AND data_type = 'jsonb' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'raw_response_text' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'parse_success' AND data_type = 'boolean' THEN '✓ CORRECT'
        WHEN column_name = 'tokens_used' AND data_type = 'integer' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'received_at' AND data_type = 'timestamp without time zone' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'llm_responses'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== LLM_RESPONSES TABLE - COLUMN STRUCTURE ===")
display(df)


In [ ]:
# =============================================================================
# CELL 9: Predictions Table - Column Structure
# =============================================================================

query = """
SELECT 
    'predictions columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name = 'prediction_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'analysis_run_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'prompt_id' AND data_type = 'integer' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'label_name' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'predicted_value' AND data_type = 'text' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'confidence_score' AND data_type = 'real' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'created_at' AND data_type = 'timestamp without time zone' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'predictions'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== PREDICTIONS TABLE - COLUMN STRUCTURE ===")
display(df)

In [ ]:
# =============================================================================
# CELL 10: Clustering Results Table - Column Structure
# =============================================================================

query = """
SELECT 
    'clustering_results columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name = 'clustering_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'analysis_run_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'image_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'cluster_id' AND data_type = 'integer' AND is_nullable = 'NO' THEN '✓ CORRECT'
        WHEN column_name = 'distance_to_centroid' AND data_type = 'real' AND is_nullable = 'YES' THEN '✓ CORRECT'
        WHEN column_name = 'embedding' AND data_type = 'text' AND is_nullable = 'YES' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'clustering_results'
ORDER BY ordinal_position;
"""

df = pd.read_sql(query, engine)
print("=== CLUSTERING_RESULTS TABLE - COLUMN STRUCTURE ===")
display(df)

In [ ]:
# =============================================================================
# CELL 11: Primary Keys Verification
# Expected: 7 primary keys (one per table)
# =============================================================================

query = """
SELECT 
    'Primary Keys' as verification_type,
    tc.table_name,
    tc.constraint_name,
    kcu.column_name,
    CASE 
        WHEN tc.table_name = 'images' AND kcu.column_name = 'image_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'ground_truth_history' AND kcu.column_name = 'history_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'prompts' AND kcu.column_name = 'prompt_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'analysis_runs' AND kcu.column_name = 'analysis_run_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'llm_responses' AND kcu.column_name = 'response_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'predictions' AND kcu.column_name = 'prediction_id' THEN '✓ CORRECT'
        WHEN tc.table_name = 'clustering_results' AND kcu.column_name = 'clustering_id' THEN '✓ CORRECT'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu 
    ON tc.constraint_name = kcu.constraint_name
    AND tc.table_schema = kcu.table_schema
WHERE tc.constraint_type = 'PRIMARY KEY'
  AND tc.table_schema = 'public'
ORDER BY tc.table_name;
"""

df = pd.read_sql(query, engine)
print("=== PRIMARY KEYS ===")
display(df)

# Summary count
query_summary = """
SELECT 
    'Primary Key Count' as metric,
    COUNT(DISTINCT tc.table_name) as tables_with_pk,
    7 as expected_count,
    CASE WHEN COUNT(DISTINCT tc.table_name) = 7 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.table_constraints tc
WHERE tc.constraint_type = 'PRIMARY KEY'
  AND tc.table_schema = 'public';
"""
df_summary = pd.read_sql(query_summary, engine)
display(df_summary)


In [ ]:
# =============================================================================
# CELL 12: Foreign Keys Verification
# Expected: 9 foreign keys with correct CASCADE/RESTRICT/SET NULL behavior
# =============================================================================

query = """
SELECT 
    'Foreign Keys' as verification_type,
    tc.table_name as from_table,
    kcu.column_name as from_column,
    ccu.table_name as to_table,
    ccu.column_name as to_column,
    rc.delete_rule,
    CASE 
        -- ground_truth_history -> images
        WHEN tc.table_name = 'ground_truth_history' AND kcu.column_name = 'image_id' 
             AND ccu.table_name = 'images' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- llm_responses -> analysis_runs
        WHEN tc.table_name = 'llm_responses' AND kcu.column_name = 'analysis_run_id' 
             AND ccu.table_name = 'analysis_runs' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- llm_responses -> images
        WHEN tc.table_name = 'llm_responses' AND kcu.column_name = 'image_id' 
             AND ccu.table_name = 'images' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- llm_responses -> prompts
        WHEN tc.table_name = 'llm_responses' AND kcu.column_name = 'prompt_id' 
             AND ccu.table_name = 'prompts' AND rc.delete_rule = 'RESTRICT' THEN '✓ CORRECT'
        -- predictions -> analysis_runs
        WHEN tc.table_name = 'predictions' AND kcu.column_name = 'analysis_run_id' 
             AND ccu.table_name = 'analysis_runs' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- predictions -> images
        WHEN tc.table_name = 'predictions' AND kcu.column_name = 'image_id' 
             AND ccu.table_name = 'images' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- predictions -> prompts
        WHEN tc.table_name = 'predictions' AND kcu.column_name = 'prompt_id' 
             AND ccu.table_name = 'prompts' AND rc.delete_rule = 'SET NULL' THEN '✓ CORRECT'
        -- clustering_results -> analysis_runs
        WHEN tc.table_name = 'clustering_results' AND kcu.column_name = 'analysis_run_id' 
             AND ccu.table_name = 'analysis_runs' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        -- clustering_results -> images
        WHEN tc.table_name = 'clustering_results' AND kcu.column_name = 'image_id' 
             AND ccu.table_name = 'images' AND rc.delete_rule = 'CASCADE' THEN '✓ CORRECT'
        ELSE '✗ CHECK DELETE RULE'
    END as status
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu 
    ON tc.constraint_name = kcu.constraint_name
    AND tc.table_schema = kcu.table_schema
JOIN information_schema.constraint_column_usage ccu 
    ON ccu.constraint_name = tc.constraint_name
    AND ccu.table_schema = tc.table_schema
JOIN information_schema.referential_constraints rc
    ON rc.constraint_name = tc.constraint_name
    AND rc.constraint_schema = tc.table_schema
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema = 'public'
ORDER BY tc.table_name, kcu.column_name;
"""

df = pd.read_sql(query, engine)
print("=== FOREIGN KEYS ===")
print("(Check delete_rule: CASCADE, RESTRICT, or SET NULL)")
display(df)

# Summary count
query_summary = """
SELECT 
    'Foreign Key Count' as metric,
    COUNT(*) as actual_count,
    9 as expected_count,
    CASE WHEN COUNT(*) = 9 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.table_constraints tc
WHERE tc.constraint_type = 'FOREIGN KEY'
  AND tc.table_schema = 'public';
"""
df_summary = pd.read_sql(query_summary, engine)
display(df_summary)



In [ ]:
# =============================================================================
# CELL 13: Unique Constraints Verification
# =============================================================================

query = """
SELECT 
    'Unique Constraints' as verification_type,
    tc.table_name,
    tc.constraint_name,
    STRING_AGG(kcu.column_name, ', ') as columns,
    CASE 
        WHEN tc.table_name = 'images' AND tc.constraint_name LIKE '%%filename%%' THEN '✓ CORRECT'
        WHEN tc.table_name = 'images' AND tc.constraint_name = 'unique_source_image' THEN '✓ CORRECT'
        WHEN tc.table_name = 'prompts' AND tc.constraint_name LIKE '%%prompt_name%%' THEN '✓ CORRECT'
        WHEN tc.table_name = 'predictions' AND tc.constraint_name = 'unique_prediction' THEN '✓ CORRECT'
        ELSE '✗ VERIFY MANUALLY'
    END as status
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu 
    ON tc.constraint_name = kcu.constraint_name
    AND tc.table_schema = kcu.table_schema
WHERE tc.constraint_type = 'UNIQUE'
  AND tc.table_schema = 'public'
GROUP BY tc.table_name, tc.constraint_name
ORDER BY tc.table_name;
"""

df = pd.read_sql(query, engine)
print("=== UNIQUE CONSTRAINTS ===")
display(df)


In [ ]:
# =============================================================================
# CELL 14: Check Constraints Verification
# Expected: check_confidence_range on predictions table
# =============================================================================

query = """
SELECT 
    'Check Constraints' as verification_type,
    tc.table_name,
    tc.constraint_name,
    cc.check_clause,
    CASE 
        WHEN tc.table_name = 'predictions' 
             AND tc.constraint_name = 'check_confidence_range' THEN '✓ CORRECT'
        ELSE '✗ VERIFY MANUALLY'
    END as status
FROM information_schema.table_constraints tc
JOIN information_schema.check_constraints cc
    ON tc.constraint_name = cc.constraint_name
    AND tc.constraint_schema = cc.constraint_schema
WHERE tc.constraint_type = 'CHECK'
  AND tc.table_schema = 'public'
ORDER BY tc.table_name;
"""

df = pd.read_sql(query, engine)
print("=== CHECK CONSTRAINTS ===")
display(df)


In [ ]:
# =============================================================================
# CELL 15: Index Verification - Part 1 (All Indexes)
# =============================================================================

query = """
SELECT 
    'Indexes' as verification_type,
    tablename,
    indexname,
    CASE 
        WHEN indexname = 'unique_current_label' THEN '✓ CRITICAL PARTIAL UNIQUE INDEX'
        WHEN indexname = 'idx_gt_history_image_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_llm_resp_analysis_run' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_llm_resp_image_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_llm_resp_prompt_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_predictions_analysis_run' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_predictions_image_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_predictions_prompt_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_clustering_analysis_run' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_clustering_image_id' THEN '✓ FK INDEX'
        WHEN indexname = 'idx_gt_current_lookup' THEN '✓ QUERY OPTIMIZATION'
        WHEN indexname = 'idx_predictions_label' THEN '✓ QUERY OPTIMIZATION'
        WHEN indexname = 'idx_predictions_lookup' THEN '✓ QUERY OPTIMIZATION'
        WHEN indexname = 'idx_analysis_runs_timestamp' THEN '✓ QUERY OPTIMIZATION'
        WHEN indexname = 'idx_analysis_runs_model' THEN '✓ QUERY OPTIMIZATION'
        WHEN indexname = 'idx_analysis_runs_hyperparams' THEN '✓ JSONB GIN INDEX'
        WHEN indexname = 'idx_llm_parsed_response' THEN '✓ JSONB GIN INDEX'
        WHEN indexname LIKE '%%_pkey' THEN '✓ AUTO-GENERATED (PK)'
        WHEN indexname = 'unique_source_image' THEN '✓ AUTO-GENERATED (UNIQUE)'
        WHEN indexname LIKE '%%_prompt_name_key' THEN '✓ AUTO-GENERATED (UNIQUE)'
        WHEN indexname LIKE '%%_filename_key' THEN '✓ AUTO-GENERATED (UNIQUE)'
        WHEN indexname = 'unique_prediction' THEN '✓ AUTO-GENERATED (UNIQUE)'
        ELSE '? VERIFY MANUALLY'
    END as status
FROM pg_indexes
WHERE schemaname = 'public'
ORDER BY tablename, indexname;
"""

df = pd.read_sql(query, engine)
print("=== ALL INDEXES ===")
print("(Includes both manually created and auto-generated indexes)")
display(df)

In [ ]:
# =============================================================================
# CELL 16: Index Verification - Part 2 (Count Check)
# Expected: 16 manually created indexes
# =============================================================================

query = """
SELECT 
    'Manually Created Index Count' as metric,
    COUNT(*) as actual_count,
    17 as expected_count,
    CASE WHEN COUNT(*) = 17 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM pg_indexes
WHERE schemaname = 'public'
  AND indexname IN (
      'unique_current_label',
      'idx_gt_history_image_id',
      'idx_llm_resp_analysis_run',
      'idx_llm_resp_image_id',
      'idx_llm_resp_prompt_id',
      'idx_predictions_analysis_run',
      'idx_predictions_image_id',
      'idx_predictions_prompt_id',
      'idx_clustering_analysis_run',
      'idx_clustering_image_id',
      'idx_gt_current_lookup',
      'idx_predictions_label',
      'idx_predictions_lookup',
      'idx_analysis_runs_timestamp',
      'idx_analysis_runs_model',
      'idx_analysis_runs_hyperparams',
      'idx_llm_parsed_response'
  );
"""

df = pd.read_sql(query, engine)
print("=== MANUALLY CREATED INDEX COUNT ===")
display(df)



In [ ]:
# =============================================================================
# CELL 17: CRITICAL - Verify Partial Unique Index
# This is the most important index - enforces business rule!
# =============================================================================

query = """
SELECT 
    'CRITICAL: Partial Unique Index' as verification_type,
    indexname,
    tablename,
    indexdef,
    CASE 
        WHEN indexdef LIKE '%%WHERE (is_current = true)%%' 
             OR indexdef LIKE '%%WHERE is_current%%' THEN '✓ CORRECT - HAS WHERE CLAUSE'
        ELSE '✗ MISSING WHERE CLAUSE - BUSINESS RULE NOT ENFORCED!'
    END as status
FROM pg_indexes
WHERE schemaname = 'public'
  AND indexname = 'unique_current_label';
"""

df = pd.read_sql(query, engine)
print("=== CRITICAL: PARTIAL UNIQUE INDEX ===")
print("This index enforces: only ONE is_current=TRUE per (image_id, label_name)")
display(df)


In [ ]:
# =============================================================================
# CELL 18: View Verification
# Expected: 7 views
# =============================================================================

query = """
SELECT 
    'Views' as verification_type,
    table_name as view_name,
    CASE 
        WHEN table_name = 'ground_truth_labels' THEN '✓ EXPECTED'
        WHEN table_name = 'ground_truth_wide' THEN '✓ EXPECTED'
        WHEN table_name = 'model_performance' THEN '✓ EXPECTED'
        WHEN table_name = 'prompt_performance' THEN '✓ EXPECTED'
        WHEN table_name = 'recent_analysis_runs' THEN '✓ EXPECTED'
        WHEN table_name = 'image_summary' THEN '✓ EXPECTED'
        WHEN table_name = 'llm_response_summary' THEN '✓ EXPECTED'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.views
WHERE table_schema = 'public'
ORDER BY table_name;
"""
df = pd.read_sql(query, engine)
print("=== VIEWS ===")
display(df)

query_summary = """
SELECT 
    'View Count' as metric,
    COUNT(*) as actual_count,
    7 as expected_count,
    CASE WHEN COUNT(*) = 7 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.views
WHERE table_schema = 'public';
"""
df_summary = pd.read_sql(query_summary, engine)
display(df_summary)


In [ ]:
# =============================================================================
# CELL 19: Sequences Verification (for SERIAL columns)
# =============================================================================

query = """
SELECT 
    'Sequences (for SERIAL columns)' as verification_type,
    sequence_name,
    data_type,
    CASE 
        WHEN sequence_name = 'images_image_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'ground_truth_history_history_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'prompts_prompt_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'analysis_runs_analysis_run_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'llm_responses_response_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'predictions_prediction_id_seq' THEN '✓ EXPECTED'
        WHEN sequence_name = 'clustering_results_clustering_id_seq' THEN '✓ EXPECTED'
        ELSE '? VERIFY MANUALLY'
    END as status
FROM information_schema.sequences
WHERE sequence_schema = 'public'
ORDER BY sequence_name;
"""
df = pd.read_sql(query, engine)
print("=== SEQUENCES (AUTO-INCREMENT FOR SERIAL COLUMNS) ===")
display(df)

In [ ]:
# =============================================================================
# CELL 20: Default Values Verification
# =============================================================================

query = """
SELECT 
    'Default Values' as verification_type,
    table_name,
    column_name,
    column_default,
    CASE 
        WHEN table_name = 'images' AND column_name = 'source' 
             AND column_default = '''giub''::text' THEN '✓ CORRECT'
        WHEN table_name = 'images' AND column_name = 'date_added' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        WHEN table_name = 'ground_truth_history' AND column_name = 'changed_at' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        WHEN table_name = 'ground_truth_history' AND column_name = 'is_current' 
             AND column_default = 'true' THEN '✓ CORRECT'
        WHEN table_name = 'prompts' AND column_name = 'created_at' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        WHEN table_name = 'analysis_runs' AND column_name = 'run_timestamp' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        WHEN table_name = 'llm_responses' AND column_name = 'parse_success' 
             AND column_default = 'false' THEN '✓ CORRECT'
        WHEN table_name = 'llm_responses' AND column_name = 'received_at' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        WHEN table_name = 'predictions' AND column_name = 'created_at' 
             AND column_default LIKE '%%now()%%' THEN '✓ CORRECT'
        ELSE '? VERIFY MANUALLY'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public'
  AND column_default IS NOT NULL
  AND table_name IN ('images', 'ground_truth_history', 'prompts', 
                     'analysis_runs', 'llm_responses', 'predictions')
ORDER BY table_name, column_name;
"""
df = pd.read_sql(query, engine)
print("=== DEFAULT VALUES ===")
display(df)

In [ ]:
# =============================================================================
# CELL 21: Migration Verification - Clustering Columns
# Verify that the migration file was applied correctly
# =============================================================================

query = """
SELECT 
    'Migration: Clustering Columns' as verification_type,
    column_name,
    data_type,
    is_nullable,
    CASE 
        WHEN column_name IN (
            'autoencoder_name', 'autoencoder_implementation', 'autoencoder_file',
            'autoencoder_params', 'dim_reduction_name', 'dim_reduction_implementation',
            'dim_reduction_params', 'clustering_name', 'clustering_implementation',
            'clustering_params'
        ) AND is_nullable = 'YES' THEN '✓ MIGRATION APPLIED'
        ELSE '✗ UNEXPECTED'
    END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'analysis_runs'
  AND column_name IN (
      'autoencoder_name', 'autoencoder_implementation', 'autoencoder_file',
      'autoencoder_params', 'dim_reduction_name', 'dim_reduction_implementation',
      'dim_reduction_params', 'clustering_name', 'clustering_implementation',
      'clustering_params'
  )
ORDER BY ordinal_position;
"""
df = pd.read_sql(query, engine)
print("=== MIGRATION VERIFICATION: CLUSTERING COLUMNS ===")
print("(4 autoencoder + 3 dim_reduction + 3 clustering = 10 columns)")
display(df)

query_count = """
SELECT 
    'Clustering Column Count' as metric,
    COUNT(*) as actual_count,
    10 as expected_count,
    CASE WHEN COUNT(*) = 10 THEN '✓ PASS' ELSE '✗ FAIL' END as status
FROM information_schema.columns
WHERE table_schema = 'public' 
  AND table_name = 'analysis_runs'
  AND column_name IN (
      'autoencoder_name', 'autoencoder_implementation', 'autoencoder_file',
      'autoencoder_params', 'dim_reduction_name', 'dim_reduction_implementation',
      'dim_reduction_params', 'clustering_name', 'clustering_implementation',
      'clustering_params'
  );
"""
df_count = pd.read_sql(query_count, engine)
display(df_count)

In [ ]:
# =============================================================================
# CELL 22: FINAL SUMMARY - Overall Schema Verification
# =============================================================================

query = """
SELECT 
    summary_section,
    metric,
    expected,
    actual,
    status
FROM (
    SELECT 1 as sort_order, 'Tables' as summary_section, 'Count' as metric, '7' as expected,
        (SELECT COUNT(*)::text FROM information_schema.tables 
         WHERE table_schema = 'public' AND table_type = 'BASE TABLE') as actual,
        CASE WHEN (SELECT COUNT(*) FROM information_schema.tables 
                   WHERE table_schema = 'public' AND table_type = 'BASE TABLE') = 7 
             THEN '✓ PASS' ELSE '✗ FAIL' END as status
    UNION ALL
    SELECT 2, 'Views', 'Count', '7',
        (SELECT COUNT(*)::text FROM information_schema.views WHERE table_schema = 'public'),
        CASE WHEN (SELECT COUNT(*) FROM information_schema.views 
                   WHERE table_schema = 'public') = 7 
             THEN '✓ PASS' ELSE '✗ FAIL' END
    UNION ALL
    SELECT 3, 'Primary Keys', 'Count', '7',
        (SELECT COUNT(DISTINCT tc.table_name)::text 
         FROM information_schema.table_constraints tc
         WHERE tc.constraint_type = 'PRIMARY KEY' AND tc.table_schema = 'public'),
        CASE WHEN (SELECT COUNT(DISTINCT tc.table_name) 
                   FROM information_schema.table_constraints tc
                   WHERE tc.constraint_type = 'PRIMARY KEY' AND tc.table_schema = 'public') = 7 
             THEN '✓ PASS' ELSE '✗ FAIL' END
    UNION ALL
    SELECT 4, 'Foreign Keys', 'Count', '9',
        (SELECT COUNT(*)::text FROM information_schema.table_constraints tc
         WHERE tc.constraint_type = 'FOREIGN KEY' AND tc.table_schema = 'public'),
        CASE WHEN (SELECT COUNT(*) FROM information_schema.table_constraints tc
                   WHERE tc.constraint_type = 'FOREIGN KEY' AND tc.table_schema = 'public') = 9 
             THEN '✓ PASS' ELSE '✗ FAIL' END
    UNION ALL
    SELECT 5, 'Manually Created Indexes', 'Count', '17',
        (SELECT COUNT(*)::text FROM pg_indexes WHERE schemaname = 'public'
         AND indexname IN (
             'unique_current_label', 'idx_gt_history_image_id',
             'idx_llm_resp_analysis_run', 'idx_llm_resp_image_id',
             'idx_llm_resp_prompt_id', 'idx_predictions_analysis_run',
             'idx_predictions_image_id', 'idx_predictions_prompt_id',
             'idx_clustering_analysis_run', 'idx_clustering_image_id',
             'idx_gt_current_lookup', 'idx_predictions_label',
             'idx_predictions_lookup', 'idx_analysis_runs_timestamp',
             'idx_analysis_runs_model', 'idx_analysis_runs_hyperparams',
             'idx_llm_parsed_response'
         )),
        CASE WHEN (SELECT COUNT(*) FROM pg_indexes WHERE schemaname = 'public'
                   AND indexname IN (
                       'unique_current_label', 'idx_gt_history_image_id',
                       'idx_llm_resp_analysis_run', 'idx_llm_resp_image_id',
                       'idx_llm_resp_prompt_id', 'idx_predictions_analysis_run',
                       'idx_predictions_image_id', 'idx_predictions_prompt_id',
                       'idx_clustering_analysis_run', 'idx_clustering_image_id',
                       'idx_gt_current_lookup', 'idx_predictions_label',
                       'idx_predictions_lookup', 'idx_analysis_runs_timestamp',
                       'idx_analysis_runs_model', 'idx_analysis_runs_hyperparams',
                       'idx_llm_parsed_response'
                   )) = 17 
             THEN '✓ PASS' ELSE '✗ FAIL' END
    UNION ALL
    SELECT 6, 'analysis_runs Columns', 'Count', '21',
        (SELECT COUNT(*)::text FROM information_schema.columns
         WHERE table_schema = 'public' AND table_name = 'analysis_runs'),
        CASE WHEN (SELECT COUNT(*) FROM information_schema.columns
                   WHERE table_schema = 'public' AND table_name = 'analysis_runs') = 21
             THEN '✓ PASS' ELSE '✗ FAIL' END
    UNION ALL
    SELECT 7, 'Clustering Migration', 'Columns Added', '10',
        (SELECT COUNT(*)::text FROM information_schema.columns
         WHERE table_schema = 'public' AND table_name = 'analysis_runs'
         AND column_name IN (
             'autoencoder_name', 'autoencoder_implementation', 'autoencoder_file',
             'autoencoder_params', 'dim_reduction_name', 'dim_reduction_implementation',
             'dim_reduction_params', 'clustering_name', 'clustering_implementation',
             'clustering_params'
         )),
        CASE WHEN (SELECT COUNT(*) FROM information_schema.columns
                   WHERE table_schema = 'public' AND table_name = 'analysis_runs'
                   AND column_name IN (
                       'autoencoder_name', 'autoencoder_implementation', 'autoencoder_file',
                       'autoencoder_params', 'dim_reduction_name', 'dim_reduction_implementation',
                       'dim_reduction_params', 'clustering_name', 'clustering_implementation',
                       'clustering_params'
                   )) = 10
             THEN '✓ PASS' ELSE '✗ FAIL' END
) summary
ORDER BY sort_order;
"""

df = pd.read_sql(query, engine)
print("=" * 60)
print("FINAL SUMMARY - SCHEMA VERIFICATION")
print("=" * 60)
display(df)
print("\n✓ If all rows show '✓ PASS', your schema matches the SQL files perfectly!")
print("✗ If any rows show '✗ FAIL', review the specific section for details.")

In [ ]:
# Close connection
cur.close()
conn.close()